In [12]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import os
import time
import json
import torch
def tic():
    return time.time()
def toc(starttime):
    return time.time()-starttime

In [13]:
model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
!pip install datasets
PATH="youcookii_annotations_trainval.json"

In [14]:
# Check if GPU is available and move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=Fals

In [17]:
counter=0
if not os.path.exists(PATH):
    !wget http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
    !tar -xvzf youcookii_annotations_trainval.tar.gz
    with open('youcookii_annotations_trainval.json', 'r') as file:
        data = json.load(file)
        data = data['database']
else:
    with open('progress.txt', 'r') as file:
        counter=int(file.read())
    with open('youcookii_annotations_trainval.json', 'r') as file:
        data = json.load(file)

--2024-12-10 08:25:20--  http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
Resolving youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)... 141.212.115.67
Connecting to youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)|141.212.115.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337048 (329K) [application/octet-stream]
Saving to: ‘youcookii_annotations_trainval.tar.gz’

youcookii_annotatio 100%[===================>] 329.15K  1.41MB/s    in 0.2s    

2024-12-10 08:25:20 (1.41 MB/s) - ‘youcookii_annotations_trainval.tar.gz’ saved [337048/337048]

youcookii_annotations_trainval.json


In [18]:
sentences = []
for item in data.values():
  annotations = item['annotations']
  for annotation in annotations:
    sentences.append(annotation['sentence'])
for i in range(10):
  print(sentences[i])

spread margarine on two slices of white bread
place a slice of cheese on the bread
place the bread slices on top of each other and place in a hot pan
flip the sandwich over and press down
cut the sandwich in half diagonally
pick the ends off the verdalago
combine lemon juice sumac garlic salt and oil in a bowl
chop lettuce and place it in a bowl
add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
pour the dressing over the salad and mix


In [19]:
def cache(translations,counter):
    with open('Translations.txt', 'a') as file:
      for string in translations:
        file.write(string + "\n")
    with open('progress.txt', 'w') as file:
        file.write(f"{counter}")

In [28]:
def add_translation(trans):
    j = 0
    for key in data.keys():
        item = data[key]
        annotations = item['annotations']
        for i in range(len(annotations)):
            if j == len(trans):
                break
            data[key]['annotations'][i]['translation'] = trans[j]
            j+=1


In [21]:
def run_model_batch(input_strings, batch_size=8,max_new_tokens=50,initial_index=0, **generator_args):
    translations = []
    counter=initial_index
    start=tic()
    trans_check=initial_index
    for i in range(initial_index, len(input_strings), batch_size):
        batch = input_strings[i:i + batch_size+1]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        # Move input tensors to GPU
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        res = model.generate(input_ids, attention_mask=attention_mask,max_new_tokens=max_new_tokens, **generator_args)
        outputs = tokenizer.batch_decode(res, skip_special_tokens=True)
        translations.extend(outputs)
        counter+=batch_size
        if(toc(start)>=10):
            start=tic()
            cache(translations[trans_check:],counter)
            trans_check= i + batch_size

    return translations

In [22]:
trans = run_model_batch(sentences[:80],initial_index=counter)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [29]:
add_translation(trans)
with open('output.json', 'w') as file:
    json.dump(data,file)

In [24]:
# len(sentences) / 80 * 16 / 60

46.09666666666667

In [ ]:
# with open("Translations.txt", "r") as file:
#     strings = file.readlines()

# # Remove the newline characters if needed
# strings = [s.strip() for s in strings]
# add_translation(strings)
# with open('output.json', 'w') as file:
#     file.write(data)